In [2]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [3]:
import numpy as np 
import pandas as pd 
import gc
import os
import json
from tqdm import tqdm_notebook
import tensorflow as tf
print(tf.__version__)
import pickle

2.1.0


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
os.chdir('/content/drive/My Drive/Colab Notebooks Shared/IDA contests/Articles RecSys')

## Read data

In [0]:
def read_json(filename):
    df = list()
    with open(filename) as f:
        for line in f:
            df.append(json.loads(line))
    return pd.DataFrame(df)

In [7]:
items_list=[]
with tqdm_notebook(open('items.json')) as inf:
    for line in inf:
        item=json.loads(line)
        if isinstance(item['image'], float):
            item['image']=[0 for _ in range(96)]
            
        item['image']=np.array(item['image'])
        items_list.append(item)

items=pd.DataFrame(items_list)

In [0]:
train = read_json('train.json')
df = list()
for index, row in train.iterrows():
    for key, value in row['trainRatings'].items():
        df.append([int(row['userId']), int(key), int(value)])
df = pd.DataFrame(df, columns = ['userId', 'itemId', 'react'])

In [8]:
del train
gc.collect()
#df.to_csv('train_user_item.csv', index=None)

0

In [8]:
#df = pd.read_csv('train_user_item.csv', index_col=0)
test = pd.read_csv('random_benchmark.csv')

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


## SVD

Получим предсказания для статей, которые когда-либо были просмотрены.

In [0]:
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

In [0]:
df['react'].replace(1, 10, inplace=True)
df['react'] = df['react'].astype(np.float32)

In [0]:
df['userId'] = df['userId'].astype(np.int32)
df['itemId'] = df['itemId'].astype(np.int32)

In [0]:
df['react'].replace(0, -1, inplace=True)

In [0]:
R = csr_matrix((df['react'], (df['userId'], df['itemId'])))

In [0]:
U, sigma, V = svds(R, k=100)

In [0]:
rg_test = test.groupby('userId')['itemId'].agg(lambda x: list(x))

In [0]:
sigma = np.diag(sigma)

In [17]:
result_svd = []
for userId, itemIds in tqdm_notebook(rg_test.iteritems(), total=len(rg_test)):
    for itemId in itemIds:
        result_svd.append({'userId': userId, 'itemId': itemId, 'react': U[userId] @ sigma @ V[:, itemId]})
result_svd = pd.DataFrame(result_svd)

In [18]:
result_svd.head()

,userId,itemId,react
0,1,242249,0.000000
1,1,117266,0.000000
2,1,123441,0.000000
3,1,148575,0.096142
4,1,155695,0.111911


In [0]:
# answ2 = result_svd.sort_values(['userId', 'react'], ascending=[True, False])
# answ2.to_csv('svd.csv', index=None)
# result_svd.to_csv('svd_score.csv', index=None)

## Predict new popular items

Обучим модель, которая предсказывает популярные статьи на основе картинок и текста даже для новых айтемов.

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
embed_size = 300 
max_features = 20000 
maxlen_text = 150 
maxlen_head = 15
maxlen_img = 96

In [0]:
mask1 = items[items['itemId'].isin(df['itemId'].values)]
#mask2 = items[items['itemId'].isin(test[test['itemNum'].isna()]['itemId'].values)]
mask3 = items[items['itemId'].isin(test['itemId'].values)]

In [0]:
df['react'].replace(-1, 0, inplace=True)

Будем считать, что статья популярна, если среднее значение просмотров >= 0.1 (это примерно 20% статей). 

In [0]:
list_sentences_train = mask1.iloc[:,0].values 
#list_sentences_test_new = mask2.iloc[:,0].values
list_sentences_test_all = mask3.iloc[:,0].values
y = df.groupby('itemId')['react'].mean()
y = (y >= 0.1).astype(np.int32)

In [0]:
list_sentences_train_head = mask1.iloc[:,3].values
#list_sentences_test_new_head = mask2.iloc[:,3].values
list_sentences_test_all_head = mask3.iloc[:,3].values

In [0]:
x_train_img = np.stack(mask1['image'].values)
#x_test_new_img = np.stack(mask2['image'].values)
x_test_all_img = np.stack(mask3['image'].values)

In [0]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
x_train_pad = pad_sequences(tokenizer.texts_to_sequences(list_sentences_train), maxlen=maxlen_text)
#x_test_pad_new = pad_sequences(tokenizer.texts_to_sequences(list_sentences_test_new), maxlen=maxlen_text)
x_test_pad_all = pad_sequences(tokenizer.texts_to_sequences(list_sentences_test_all), maxlen=maxlen_text)

In [0]:
x_train_pad_head = pad_sequences(tokenizer.texts_to_sequences(list_sentences_train_head), maxlen=maxlen_head)
#x_test_pad_new_head = pad_sequences(tokenizer.texts_to_sequences(list_sentences_test_new_head), maxlen=maxlen_head)
x_test_pad_all_head = pad_sequences(tokenizer.texts_to_sequences(list_sentences_test_all_head), maxlen=maxlen_head)

In [0]:
x_all_content = pad_sequences(tokenizer.texts_to_sequences(items['content'].values), maxlen=maxlen_text)
x_all_head = pad_sequences(tokenizer.texts_to_sequences(items['title'].values), maxlen=maxlen_head)
x_all_img = np.stack(items['image'].values)

In [0]:
from tensorflow.keras.models import save_model, load_model

class ModelEndCallback(tf.keras.callbacks.Callback):

    def __init__(self, file_name):
        super(ModelEndCallback, self).__init__()
        self.file_name = file_name

    def on_epoch_end(self, epoch, logs=None):
        save_model(self.model, self.file_name.format(epoch+1))

def load_from_file(model_filename, last_epoch):
    return load_model(model_filename.format(last_epoch))

Эмбеддинг взят с [сайта](https://fasttext.cc/docs/en/crawl-vectors.html)

In [31]:
f = open('./cc.ru.300.vec')
embeddings_index = dict() 
for line in tqdm_notebook(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [32]:
all_embs = np.stack(embeddings_index.values())
emb_mean, emb_std = all_embs.mean(), all_embs.std()
emb_mean, emb_std

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):


(-7.195237e-05, 0.054040156)

In [0]:
word_index = tokenizer.word_index
unknown_words = set()
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    else: unknown_words.add(word)

In [0]:
from tensorflow.keras.layers import Dense, Input, GRU, Embedding, Dropout, Bidirectional, Flatten, LeakyReLU, BatchNormalization, LSTM, SpatialDropout1D, Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler

In [0]:
def make_lstm(input_layer, units):
    embedding = Embedding(max_features, embed_size, 
                          weights=[embedding_matrix], 
                          trainable = True)(input_layer)
    x = Bidirectional(LSTM(units, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(embedding)
    x1 = GlobalMaxPooling1D()(x)
    x2 = GlobalAveragePooling1D()(x)
    x = concatenate([x1, x2])   
    return x

def make_model(maxlen_text, maxlen_head):
    initializer = tf.keras.initializers.glorot_normal()
    input_text = Input((maxlen_text,), name = '0')
    input_head = Input((maxlen_head,), name = '1')
    input_img = Input((maxlen_img,), name = '2')
    x1 = make_lstm(input_text, 80)  
    x2 = make_lstm(input_head, 10)           
    x3 = Dense(64, activation='relu')(input_img)

    x = concatenate([x1, x2, x3])
    x = Dense(80, activation="relu", name="last_layer")(x)
    x = Dropout(0.1)(x)
    output_layer = Dense(1, activation="sigmoid")(x)

    model = Model(inputs=[input_text, input_head, input_img], outputs=output_layer)
    model.compile(loss='binary_crossentropy',
                    optimizer=Adam(clipnorm=2, clipvalue=2),
                    metrics=['accuracy', 'AUC'])
    embed_model = Model(inputs=model.input, outputs=model.get_layer("last_layer").output)
    return model, embed_model

def schedule(ind):
    a = [0.001, 0.001, 0.0001, 0.0001, 0.00001, 0.00001, 0.000001]
    return a[ind]

lr = LearningRateScheduler(schedule)

In [36]:
model_filename = '00_masha_text_weights_{0:02d}.hdf5'
model, embed_model = make_model(maxlen_text, maxlen_head)
history = model.fit([x_train_pad, x_train_pad_head, x_train_img], y, batch_size=512, epochs = 2, 
                    validation_split = 0.1, verbose = 1, callbacks = [lr, ModelEndCallback(model_filename)])

Train on 218120 samples, validate on 24236 samples
Epoch 1/2
218120/218120 [==============================] - 278s 1ms/sample - loss: 0.4103 - accuracy: 0.8459 - AUC: 0.6473 - val_loss: 0.4005 - val_accuracy: 0.8444 - val_AUC: 0.6939
Epoch 2/2
218120/218120 [==============================] - 268s 1ms/sample - loss: 0.3678 - accuracy: 0.8513 - AUC: 0.7609 - val_loss: 0.4081 - val_accuracy: 0.8430 - val_AUC: 0.6864


In [37]:
embed_data = embed_model.predict([x_all_content, x_all_head, x_all_img], batch_size=512*8, verbose=1)

328050/328050 [==============================] - 26s 78us/sample


In [0]:
#embed_data.dump('00_masha_embedding.pkl')

In [39]:
target1 = model.predict([x_test_pad_all, x_test_pad_all_head,  x_test_all_img], batch_size=512, verbose=1)
#target2 = model.predict([x_test_pad_new, x_test_pad_new_head, x_test_new_img], batch_size=512, verbose=1)

142491/142491 [==============================] - 31s 217us/sample


In [0]:
ids1 = items[items['itemId'].isin(test['itemId'].values)].iloc[:,1].values
#ids2 = items[items['itemId'].isin(test[test['itemNum'].isna()]['itemId'].values)].iloc[:,1].values

In [0]:
res1 = pd.DataFrame(data={'itemId': ids1, 'react': target1[:, 0]})
#res1.to_csv('00_masha_pop_all.csv', index=None)
#res2 = pd.DataFrame(data={'itemId': ids2, 'react': target2[:, 0]})
#res2.to_csv('00_masha_pop_new.csv', index=None)

In [42]:
res1.head()

,itemId,react
0,3,0.935964
1,5,0.905945
2,6,0.935081
3,7,0.933901
4,8,0.940302


## Classifer on embedding

На полученном эмбеддинге из предыдущей модели обучим классификатор для каждого пользователя.

In [0]:
#rg_test = test.groupby('userId')['itemId'].agg(lambda x: list(x))
rg_train = df.groupby('userId')[['itemId', 'react']].agg(lambda x: list(x))
rg_all = rg_train.join(rg_test, on='userId', how='inner', rsuffix='_test')

In [0]:
from sklearn.linear_model import SGDClassifier

In [0]:
result = []
for userId, row in tqdm_notebook(rg_all.iterrows(), total=len(rg_all)):
    item_train = row['itemId']
    react_train = np.array(row['react']).astype(np.uint8)
    item_test = row['itemId_test']

    data_train = embed_data[item_train]
    data_test = embed_data[item_test]

    clf = SGDClassifier(loss='log', random_state=42)
    clf.fit(data_train, react_train)
    proba = clf.predict_proba(data_test)[:, 1]

    result.extend(zip([userId] * len(proba), item_test, proba))
    # result += list(zip([userId] * len(item_test), item_test, proba))
result = pd.DataFrame(result)
result.columns = ['userId', 'itemId', 'react']

In [0]:
#result.to_pickle('00_masha_embedding_sgd_presubmit.pkl')

## Concatenate results

По отдельности все модели показывают среднее качество, но если объединить предсказания (это мои лучшие модели по качеству, которые удалось обучить), то можно хорошо подняться в лидерборде. Суммирование нормированных предсказаний не дало такого прироста качества, как ранжирование статей по сумме их порядковых нормеров из всех предсказаний.

In [0]:
res1.set_index('itemId', inplace=True)

In [0]:
result_nn = test.join(res1, on='itemId')[['userId', 'itemId', 'react']]

In [0]:
result.set_index(['userId', 'itemId'], inplace=True)
result_svd.set_index(['userId', 'itemId'], inplace=True)
result_nn.set_index(['userId', 'itemId'], inplace=True)

In [0]:
result_full = result.join(result_svd, on=['userId', 'itemId'],rsuffix='_svd').join(result_nn, on=['userId', 'itemId'],rsuffix='_nn')

In [0]:
#result_full.to_pickle('00_masha_sgd_svd_nn_raw_full.pkl')

In [0]:
result_full['react_svd_norm'] = (result_full['react_svd'] -  result_full['react_svd'].min())/(result_full['react_svd'].max() -  result_full['react_svd'].min())

In [0]:
result_full['react_nn_norm'] = (result_full['react_nn'] -  result_full['react_nn'].min())/(result_full['react_nn'].max() -  result_full['react_nn'].min())

In [0]:
result_full['react_sum'] = result_full['react'] + result_full['react_svd_norm'] + result_full['react_nn_norm']

In [0]:
result_full.reset_index(inplace=True)

In [0]:
result_full

,userId,itemId,react,react_svd,react_nn,react_svd_norm,react_nn_norm,react_sum
0,1,242249,0.180505,0.000000,0.255616,0.476692,0.284036,0.941233
1,1,117266,0.283809,0.000000,0.154825,0.476692,0.170319,0.930820
2,1,123441,0.189976,0.000000,0.221664,0.476692,0.245730,0.912398
3,1,148575,0.250512,0.096142,0.104330,0.482289,0.113348,0.846149
4,1,155695,0.244766,0.111911,0.179998,0.483207,0.198720,0.926693
...,...,...,...,...,...,...,...,...
3018181,42975,4202,0.059244,0.006989,0.496445,0.477099,0.555751,1.092094
3018182,42975,111136,0.021439,0.000000,0.299767,0.476692,0.333850,0.831981
3018183,42975,195676,0.026460,0.000000,0.099942,0.476692,0.108398,0.611549
3018184,42975,292604,0.016078,-0.006573,0.390884,0.476309,0.436652,0.929040


In [0]:
#answer = result_full.sort_values(['userId', 'react_nn'], ascending=[True, False])
#answer[['userId', 'itemId']].to_csv('00_masha_nn.csv', index=None)

In [0]:
answer = result_full.sort_values(['userId', 'react_sum'], ascending=[True, False])
#answer[['userId', 'itemId']].to_csv('00_masha_full.csv', index=None)

In [0]:
#result_full = pd.read_pickle('00_masha_full.pkl')

In [0]:
df1 = result_full[['react']].sort_values(['userId', 'react'], ascending=[True, False])
df2 = result_full[['react_svd']].sort_values(['userId', 'react_svd'], ascending=[True, False])
df3 = result_full[['react_nn']].sort_values(['userId', 'react_nn'], ascending=[True, False])

In [0]:
df1.reset_index(inplace=True)
df2.reset_index(inplace=True)
df3.reset_index(inplace=True)

In [0]:
df1['index'] = df1.index
df2['index'] = df2.index
df3['index'] = df3.index

In [0]:
df1 = df1.groupby(['userId', 'itemId'])['index'].mean()
df2 = df2.groupby(['userId', 'itemId'])['index'].mean()
df3 = df3.groupby(['userId', 'itemId'])['index'].mean()

In [0]:
df1 = pd.DataFrame(df1)
df2 = pd.DataFrame(df2)
df3 = pd.DataFrame(df3)

In [0]:
df = df1.join(df2, on=['userId', 'itemId'],rsuffix='_2').join(df3, on=['userId', 'itemId'], rsuffix='_3')

In [0]:
df.head()

index   index_2   index_3
userId itemId                              
1      307     0.000105  0.000044  0.000196
       788     0.000101  0.000156  0.000153
       1397    0.000091  0.000011  0.000095
       1440    0.000137  0.000142  0.000093
       1610    0.000056  0.000191  0.000078

In [0]:
df['react'] = df['index'] + df['index_2']  + df['index_3']

In [0]:
df.reset_index(inplace=True)

In [0]:
df.sort_values(['userId', 'react'], ascending=[True, True])[['userId', 'itemId']].to_csv('final_answer.csv', index=None)